# **Scenario 2.**

A retail company receives daily sales files from multiple stores in an Azure Data Lake folder. Instead of reprocessing all the historical data every day, the data engineering team uses Spark Structured Streaming to load only the new files into a Delta table. This keeps the analytics dashboards up to date while reducing compute costs and processing time.

### **Streaming Query**


**Define Schema**

In [0]:

my_schema = """
    order_id INT,
    customer_id INT,
    order_date DATE,
    amount DOUBLE  
"""

**Read in batch mode first**

In [0]:
df_batch = spark.read.format("csv")\
        .option("header", "true")\
        .schema(my_schema)\
        .load("/Volumes/pyspark_catalog/source/db_volume/stream_source/")

display(df_batch)        

**Read Data in stream**

In [0]:
df = spark.readStream.format("csv")\
        .option("header", "true")\
        .schema(my_schema)\
        .load("/Volumes/pyspark_catalog/source/db_volume/stream_source/")

### **Streaming Output**

In [0]:
df.writeStream.format("delta")\
        .option("checkpointLocation", "/Volumes/pyspark_catalog/source/db_volume/stream_sink/checkpoint/")\
        .option("mergeSchema", True)\
        .trigger(once=True)\
        .start("/Volumes/pyspark_catalog/source/db_volume/stream_sink/data")
                 


In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_catalog/source/db_volume/stream_sink/data/`

**Resule after adding new file**

In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_catalog/source/db_volume/stream_sink/data/`